## Train Xception Model to recognize the Intended Gender for a Watch

In [1]:

import os

import numpy as np
import pandas as pd

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dropout, Flatten, Dense,GlobalAveragePooling2D
from keras import applications
#from keras import backend as K
from keras import callbacks

Using TensorFlow backend.


In [2]:
# dimensions of our images.
img_width, img_height = 299, 299

# dimensions of our images.
img_width, img_height = 299, 299
n_features = 1

train_data_dir = 'binary_gender_masked/train/'
validation_data_dir = 'binary_gender_masked/test/'

available_train_files = len(os.listdir(train_data_dir + 'female/')) \
    + len(os.listdir(train_data_dir + 'male/'))
available_test_files = len(os.listdir(validation_data_dir + 'female/')) \
    + len(os.listdir(validation_data_dir+'male/'))
    
nb_train_samples  = available_train_files - available_train_files % 8
nb_validation_samples = available_test_files - available_test_files % 8

batch_size = 8

In [3]:
available_train_files % 8

0

Create Generators

In [4]:
datagen = ImageDataGenerator(
        rescale=1. / 255,
        rotation_range=90,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    shuffle=True,
    class_mode = 'binary')

test_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    shuffle=True,
    class_mode = 'binary')

Found 7368 images belonging to 2 classes.
Found 3159 images belonging to 2 classes.


Load Standard Xception Model

In [9]:
model = applications.xception.Xception(include_top=False, weights='imagenet')

Build My "Custom" Model

In [10]:
watch_model = Sequential()
watch_model.add(model)
watch_model.add(GlobalAveragePooling2D(name='avg_pool'))
watch_model.add(Dense(1, activation="sigmoid"))
watch_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, None, None, 2048)  20861480  
_________________________________________________________________
avg_pool (GlobalAveragePooli (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2049      
Total params: 20,863,529
Trainable params: 20,809,001
Non-trainable params: 54,528
_________________________________________________________________


Freeze Exception Layers

In [11]:
for layer in watch_model.layers[0].layers:
    layer.trainable = False

Compile

In [12]:
watch_model.compile(
    loss = "binary_crossentropy", 
    optimizer='sgd', 
    metrics=["binary_accuracy"])

In [13]:
#Fit 

epochs = 5

cb1 = [callbacks.ModelCheckpoint(
    'xception_binary_gender_masked_best1.h5',
    monitor='val_loss',
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False, 
    mode='auto', period=1)]

# fine-tune the model
watch_model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data = test_generator,
        validation_steps=nb_validation_samples // batch_size,
        callbacks = cb1)

watch_model.save('xception_binary_gender_masked_final1.h5')

Epoch 1/1
921/921 [==============================] - 453s - loss: 0.5498 - binary_accuracy: 0.7288 - val_loss: 0.4898 - val_binary_accuracy: 0.7773


In [14]:
del watch_model

In [15]:
watch_model = load_model('xception_binary_gender_masked_final1.h5')

In [16]:
#Unfreeze a few more layers and allow to run

# watch_model.layers[0].summary()

# Freeze convolutional layers
for layer in watch_model.layers[0].layers[-3:]:
    layer.trainable = True

epochs = 20


watch_model.compile(
    loss = "binary_crossentropy", 
    optimizer='sgd', 
    metrics=["binary_accuracy"])

# watch_model.layers[0].summary()

cb2 = [callbacks.ModelCheckpoint(
    'xception_binary_gender_masked_best2.h5',
    monitor='val_loss',
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False, 
    mode='auto', period=1)]

# fine-tune the model
watch_model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data = test_generator,
        validation_steps=nb_validation_samples // batch_size,
        callbacks = cb2)


watch_model.save('xception_binary_gender_masked_final2.h5')

Epoch 1/1
921/921 [==============================] - 467s - loss: 0.4807 - binary_accuracy: 0.7709 - val_loss: 0.4382 - val_binary_accuracy: 0.7997


In [5]:
watch_model = load_model('xception_binary_gender_masked_final2.h5')

In [6]:
#Unfreeze a few more layers and allow to run

# watch_model.layers[0].summary()

# # Freeze convolutional layers
# for layer in watch_model.layers[0].layers[-3:]:
#     layer.trainable = True

epochs = 10


watch_model.compile(
    loss = "binary_crossentropy", 
    optimizer='sgd', 
    metrics=["binary_accuracy"])

watch_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, None, None, 2048)  20861480  
_________________________________________________________________
avg_pool (GlobalAveragePooli (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2049      
Total params: 20,863,529
Trainable params: 3,165,697
Non-trainable params: 17,697,832
_________________________________________________________________


In [ ]:
cb3 = [callbacks.ModelCheckpoint(
    'xception_binary_gender_masked_best3.h5',
    monitor='val_loss',
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False, 
    mode='auto', period=1)]

# fine-tune the model
watch_model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data = test_generator,
        validation_steps=nb_validation_samples // batch_size,
        callbacks = cb3)


watch_model.save('xception_binary_gender_masked_final3.h5')

Epoch 1/10
921/921 [==============================] - 467s - loss: 0.3190 - binary_accuracy: 0.8609 - val_loss: 0.2968 - val_binary_accuracy: 0.8788
Epoch 2/10
111/921 [==>...........................] - ETA: 276s - loss: 0.3422 - binary_accuracy: 0.8604